In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy.sparse as sp
from torch.utils.data import Dataset, DataLoader

In [ ]:
class BasicDataset(Dataset):
    def __init__(self):
        print("init dataset")
    
    @property
    def n_users(self):
        raise NotImplementedError
    
    @property
    def m_items(self):
        raise NotImplementedError
    
    @property
    def trainDataSize(self):
        raise NotImplementedError
    
    @property
    def testDict(self):
        raise NotImplementedError
    
    @property
    def allPos(self):
        raise NotImplementedError
    
    def getUserItemFeedback(self, users, items):
        raise NotImplementedError
    
    def getUserPosItems(self, users):
        raise NotImplementedError
    
    def getUserNegItems(self, users):
        """
        not necessary for large dataset
        it's stupid to return all neg items in super large dataset
        """
        raise NotImplementedError
    
    def getSparseGraph(self):
        """
        build a graph in torch.sparse.IntTensor.
        Details in NGCF's matrix form
        A = 
            |I,   R|
            |R^T, I|
        """
        raise NotImplementedError

In [ ]:
import csv
csv_file = input('ml100k/movies.csv')
txt_file = input('ml100k/movies_list.txt')
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

In [31]:
import csv
csv_file = input('ml100k/ratings.csv')
txt_file = input('ml100k/train.txt')
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

SyntaxError: can't assign to operator (<ipython-input-31-ea513b44d9cb>, line 6)

In [ ]:
txt_file = input('ml100k/user_list.txt')
with open(txt_file, "w") as my_output_file:
    for i in range(609):
        
        [ my_output_file.write("user_"+ str(i+1) +" "+ str(i+1) +'\n')]
    my_output_file.close()

In [46]:
train_file = '/Users/sayamsingla/Desktop/Warwick/LightGCN-PyTorch-master/data/gowalla/train.txt'
train_file2 = '/Users/sayamsingla/Desktop/Warwick/LightGCN-PyTorch-master/data/ml100k/train.txt'
path = '/Users/sayamsingla/Desktop/Warwick/LightGCN-PyTorch-master/code/ml100k'

In [66]:
trainUniqueUsers, trainItem, trainUser , trainRating= [], [], [], []
testUniqueUsers, testItem, testUser = [], [], []
trainDataSize = 0
testDataSize = 0
n_user = 0
m_item = 0
ref = 0
c = []

with open(train_file2) as f:
            for l in f.readlines():
                
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    #items = [int(i) for i in l[1]]
                    items = int(l[1])
                    rating = float(str(l[2])) 
                    
                    
                    uid = int(l[0])
                    
                    if uid > ref:
                        trainUniqueUsers.append(uid)
                        ref = uid
                    
                    trainRating.append(rating)
                    #trainUser.extend([uid] * len(items))
                    trainUser.append(uid)
                    #trainItem.extend(items)
                    trainItem.append(items)
                    m_item = max(m_item, items)
                    n_user = max(n_user, uid)
                    trainDataSize += 1
c = trainItem
m_item = len(np.array(list(set(c))))

trainUniqueUsers = np.array(trainUniqueUsers)
trainUser = np.array(trainUser)
trainItem = np.array(trainItem)
trainRating = np.array(trainRating)
m_item = trainItem.max()


In [67]:
size = trainItem.max()
trainUser, trainItem, m_item

(array([  1,   1,   1, ..., 610, 610, 610]),
 array([     1,      3,      6, ..., 168250, 168252, 170875]),
 193609)

In [49]:
trainItem.shape , trainUser.shape , trainRating.shape , m_item, n_user , trainDataSize

((100836,), (100836,), (100836,), 193609, 610, 100836)

In [64]:
trainUniqueUsers, trainItem, trainUser = [], [], []
testUniqueUsers, testItem, testUser = [], [], []
trainDataSize = 0
testDataSize = 0
n_user = 0
m_item = 0
c =   0
with open(train_file) as f:
            for l in f.readlines():
                c +=1
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    trainUniqueUsers.append(uid)
                    trainUser.extend([uid] * len(items))
                    trainItem.extend(items)
                    m_item = max(m_item, max(items))
                    n_user = max(n_user, uid)
                    trainDataSize += len(items)
                    
trainUniqueUsers = np.array(trainUniqueUsers)
trainUser = np.array(trainUser)
trainItem = np.array(trainItem)
c

29858

In [65]:
trainUser, trainItem


(array([    0,     0,     0, ..., 29857, 29857, 29857]),
 array([   0,    1,    2, ..., 1853,  691,  674]))

In [59]:
trainItem.shape, len(trainUser)

((810128,), 810128)

In [6]:
def getUserPosItems( users):
        posItems = []
        for user in users:
            posItems.append(UserItemNet[user].nonzero()[1])
        return posItems

In [69]:
        Graph = None
        print(f"{trainDataSize} interactions for training")
        print(f"{testDataSize} interactions for testing")
        print(f"Sparsity : {(trainDataSize + testDataSize) / n_user / m_item}")
        m_item += 1
        n_user += 1
        #print(n_user, size)
        # (users,items), bipartite graph
       # UserItemNet = csr_matrix((np.ones(len(trainUser)), (trainUser, trainItem)),shape=(n_user, m_item))
        UserItemNet = csr_matrix((trainRating, (trainUser, trainItem)),shape=(n_user, m_item))
        print(UserItemNet)
        
        users_D = np.array(UserItemNet.sum(axis=1)).squeeze()
        users_D[users_D == 0.] = 1
        items_D = np.array(UserItemNet.sum(axis=0)).squeeze()
        items_D[items_D == 0.] = 1.
        # pre-calculate
        _allPos = getUserPosItems(list(range(n_user)))
        #__testDict = __build_test()
        

100836 interactions for training
0 interactions for testing
Sparsity : 0.0008524062284253587
  (1, 1)	4.0
  (1, 3)	4.0
  (1, 6)	4.0
  (1, 47)	5.0
  (1, 50)	5.0
  (1, 70)	3.0
  (1, 101)	5.0
  (1, 110)	4.0
  (1, 151)	5.0
  (1, 157)	5.0
  (1, 163)	5.0
  (1, 216)	5.0
  (1, 223)	3.0
  (1, 231)	5.0
  (1, 235)	4.0
  (1, 260)	5.0
  (1, 296)	3.0
  (1, 316)	3.0
  (1, 333)	5.0
  (1, 349)	4.0
  (1, 356)	4.0
  (1, 362)	5.0
  (1, 367)	4.0
  (1, 423)	3.0
  (1, 441)	4.0
  :	:
  (610, 156371)	5.0
  (610, 156726)	4.5
  (610, 157296)	4.0
  (610, 158238)	5.0
  (610, 158721)	3.5
  (610, 158872)	3.5
  (610, 158956)	3.0
  (610, 159093)	3.0
  (610, 160080)	3.0
  (610, 160341)	2.5
  (610, 160527)	4.5
  (610, 160571)	3.0
  (610, 160836)	3.0
  (610, 161582)	4.0
  (610, 161634)	4.0
  (610, 162350)	3.5
  (610, 163937)	3.5
  (610, 163981)	3.5
  (610, 164179)	5.0
  (610, 166528)	4.0
  (610, 166534)	4.0
  (610, 168248)	5.0
  (610, 168250)	5.0
  (610, 168252)	5.0
  (610, 170875)	3.0


In [27]:
n_users = n_user
m_items = m_item


In [29]:
        if Graph is None:
            try:
                pre_adj_mat = sp.load_npz(path + '/s_pre_adj_mat.npz')
                print("successfully loaded...")
                norm_adj = pre_adj_mat
            except :
                print("generating adjacency matrix")
                #s = time()
                adj_mat = sp.dok_matrix((n_users + m_items, n_users + m_items), dtype=np.float32)
                adj_mat = adj_mat.tolil()
                R = UserItemNet.tolil()
                adj_mat[:n_users, n_users:] = R
                adj_mat[n_users:, :n_users] = R.T
                adj_mat = adj_mat.todok()
                # adj_mat = adj_mat + sp.eye(adj_mat.shape[0])
                
                rowsum = np.array(adj_mat.sum(axis=1))
                d_inv = np.power(rowsum, -0.5).flatten()
                d_inv[np.isinf(d_inv)] = 0.
                d_mat = sp.diags(d_inv)
                
                norm_adj = d_mat.dot(adj_mat)
                norm_adj = norm_adj.dot(d_mat)
                norm_adj = norm_adj.tocsr()
                #end = time()
                print(f"costing s, saved norm_mat...")
                sp.save_npz(path + '/s_pre_adj_mat.npz', norm_adj)

            if self.split == True:
                self.Graph = _split_A_hat(norm_adj)
                print("done split matrix")
            else:
                self.Graph = self._convert_sp_mat_to_sp_tensor(norm_adj)
                self.Graph = self.Graph.coalesce().to(world.device)
                print("don't split the matrix")

generating adjacency matrix
costing s, saved norm_mat...


/Users/sayamsingla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in power


NameError: name 'self' is not defined

In [ ]:
class Loader(BasicDataset):
    """
    Dataset type for pytorch \n
    Incldue graph information
    gowalla dataset
    """

    def __init__(self,path="../data/gowalla"):
        # train or test
       
        self.mode_dict = {'train': 0, "test": 1}
        self.mode = self.mode_dict['train']
        self.n_user = 0
        self.m_item = 0
        train_file = path + '/train.txt'
        test_file = path + '/test.txt'
        self.path = path
        trainUniqueUsers, trainItem, trainUser = [], [], []
        testUniqueUsers, testItem, testUser = [], [], []
        self.traindataSize = 0
        self.testDataSize = 0

        with open(train_file) as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    trainUniqueUsers.append(uid)
                    trainUser.extend([uid] * len(items))
                    trainItem.extend(items)
                    self.m_item = max(self.m_item, max(items))
                    self.n_user = max(self.n_user, uid)
                    self.traindataSize += len(items)
        self.trainUniqueUsers = np.array(trainUniqueUsers)
        self.trainUser = np.array(trainUser)
        self.trainItem = np.array(trainItem)

        with open(test_file) as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    testUniqueUsers.append(uid)
                    testUser.extend([uid] * len(items))
                    testItem.extend(items)
                    self.m_item = max(self.m_item, max(items))
                    self.n_user = max(self.n_user, uid)
                    self.testDataSize += len(items)
        self.m_item += 1
        self.n_user += 1
        self.testUniqueUsers = np.array(testUniqueUsers)
        self.testUser = np.array(testUser)
        self.testItem = np.array(testItem)
        
        self.Graph = None
        print(f"{self.trainDataSize} interactions for training")
        print(f"{self.testDataSize} interactions for testing")
        print(f"Sparsity : {(self.trainDataSize + self.testDataSize) / self.n_users / self.m_items}")

        # (users,items), bipartite graph
        self.UserItemNet = csr_matrix((np.ones(len(self.trainUser)), (self.trainUser, self.trainItem)),
                                      shape=(self.n_user, self.m_item))
        print(self.n_user, self.m_item)
        self.users_D = np.array(self.UserItemNet.sum(axis=1)).squeeze()
        self.users_D[self.users_D == 0.] = 1
        self.items_D = np.array(self.UserItemNet.sum(axis=0)).squeeze()
        self.items_D[self.items_D == 0.] = 1.
        # pre-calculate
        self._allPos = self.getUserPosItems(list(range(self.n_user)))
        self.__testDict = self.__build_test()
        

    @property
    def n_users(self):
        return self.n_user
    
    @property
    def m_items(self):
        return self.m_item
    
    @property
    def trainDataSize(self):
        return self.traindataSize
    
    @property
    def testDict(self):
        return self.__testDict

    @property
    def allPos(self):
        return self._allPos

    def _split_A_hat(self,A):
        A_fold = []
        fold_len = (self.n_users + self.m_items) // self.folds
        for i_fold in range(self.folds):
            start = i_fold*fold_len
            if i_fold == self.folds - 1:
                end = self.n_users + self.m_items
            else:
                end = (i_fold + 1) * fold_len
            A_fold.append(self._convert_sp_mat_to_sp_tensor(A[start:end]).coalesce().to(world.device))
        return A_fold

    def _convert_sp_mat_to_sp_tensor(self, X):
        coo = X.tocoo().astype(np.float32)
        row = torch.Tensor(coo.row).long()
        col = torch.Tensor(coo.col).long()
        index = torch.stack([row, col])
        data = torch.FloatTensor(coo.data)
        return torch.sparse.FloatTensor(index, data, torch.Size(coo.shape))
        
    def getSparseGraph(self):
        print("loading adjacency matrix")
        if self.Graph is None:
            try:
                pre_adj_mat = sp.load_npz(self.path + '/s_pre_adj_mat.npz')
                print("successfully loaded...")
                norm_adj = pre_adj_mat
            except :
                print("generating adjacency matrix")
                s = time()
                adj_mat = sp.dok_matrix((self.n_users + self.m_items, self.n_users + self.m_items), dtype=np.float32)
                adj_mat = adj_mat.tolil()
                R = self.UserItemNet.tolil()
                adj_mat[:self.n_users, self.n_users:] = R
                adj_mat[self.n_users:, :self.n_users] = R.T
                adj_mat = adj_mat.todok()
                # adj_mat = adj_mat + sp.eye(adj_mat.shape[0])
                
                rowsum = np.array(adj_mat.sum(axis=1))
                d_inv = np.power(rowsum, -0.5).flatten()
                d_inv[np.isinf(d_inv)] = 0.
                d_mat = sp.diags(d_inv)
                
                norm_adj = d_mat.dot(adj_mat)
                norm_adj = norm_adj.dot(d_mat)
                norm_adj = norm_adj.tocsr()
                end = time()
                print(f"costing {end-s}s, saved norm_mat...")
                sp.save_npz(self.path + '/s_pre_adj_mat.npz', norm_adj)

            if self.split == True:
                self.Graph = self._split_A_hat(norm_adj)
                print("done split matrix")
            else:
                self.Graph = self._convert_sp_mat_to_sp_tensor(norm_adj)
                self.Graph = self.Graph.coalesce().to(world.device)
                print("don't split the matrix")
        return self.Graph

    def __build_test(self):
        """
        return:
            dict: {user: [items]}
        """
        test_data = {}
        for i, item in enumerate(self.testItem):
            user = self.testUser[i]
            if test_data.get(user):
                test_data[user].append(item)
            else:
                test_data[user] = [item]
        return test_data

    def getUserItemFeedback(self, users, items):
        """
        users:
            shape [-1]
        items:
            shape [-1]
        return:
            feedback [-1]
        """
        # print(self.UserItemNet[users, items])
        return np.array(self.UserItemNet[users, items]).astype('uint8').reshape((-1,))

    def getUserPosItems(self, users):
        posItems = []
        for user in users:
            posItems.append(self.UserItemNet[user].nonzero()[1])
        return posItems

    # def getUserNegItems(self, users):
    #     negItems = []
    #     for user in users:
    #         negItems.append(self.allNeg[user])
    #     return negItems

In [ ]:
 l = Loader()